**TODO**:
* calibration : find the threshold value to select the right channel (meld 2 channels)
* rotation before Tino's geometry transformation ?
* optimize parameters for noise injection

In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 10)

import numpy as np
#np.set_printoptions(threshold=np.nan)
from matplotlib import pyplot as plt
from matplotlib import cm

import ctapipe
from ctapipe.io.hessio import hessio_event_source
import pyhessio

import sys
sys.path.append("../")

import datapipe
import datapipe.denoising.wavelets_mrfilter as mrfilter

sys.path.append("../utils")
import common_functions as common

from datapipe.io import images
from datapipe.io import geometry_converter

Nectarcam = "NectarCam" (tel 5 to 16 in prod3), 2 channels

In [ ]:
%run ~/git/pub/jdhp-sap/snippets/ctapipe/list_telescopes_geometry.py ~/vm-shared/data/prod3/full_telarray_example.simtel.gz

In [ ]:
#ctapipe.version.get_version()
print(ctapipe.__version__)
print(pyhessio.__version__)

import ctapipe.image.geometry_converter as geomconv
from ctapipe.instrument import CameraGeometry

In [ ]:
# calibrator
from ctapipe.calib import CameraCalibrator

calib = CameraCalibrator(None, None)

In [ ]:
tel_id = 5
source = hessio_event_source("/Users/jdecock/vm-shared/data/prod3/full_telarray_example.simtel.gz", allowed_tels=[tel_id])

In [ ]:
for ev in source:
    calib.calibrate(ev)  # calibrate the event
    break

In [ ]:
print(ev)

In [ ]:
# GET IMAGES ##############################################

channel = 0

pe = ev.mc.tel[tel_id].photo_electron_image
pedestal = ev.mc.tel[tel_id].pedestal[channel]
gain = ev.mc.tel[tel_id].dc_to_pe[channel]
uncalibrated_image = ev.r0.tel[tel_id].adc_sums[channel]

calibrated_image = ev.dl1.tel[tel_id].image

## Mix channels (not done in ctapipe because cameras will do it automatically in production, there will be only 1 channel then ; i.e. this issue only concerns simulations)
## Taken from TinoCTA ./modules/prepare_event.py l.77
#pmt_signal = event.dl1.tel[tel_id].image
#if pmt_signal.ndim > 1 and pmt_signal.shape[0] > 1:
#    pick = (pmt_signal > 14).any(axis=0) != np_true_false
#    pmt_signal = pmt_signal.T[pick.T]
#else:
#    pmt_signal = pmt_signal.ravel()
threshold = 200
calibrated_image[1, calibrated_image[0,:] <= threshold] = 0
calibrated_image[0, calibrated_image[0,:] >  threshold] = 0
calibrated_image = calibrated_image.sum(axis=0)

pixel_pos = ev.inst.pixel_pos[tel_id]

event_id = ev.dl0.event_id

In [ ]:
# %load ~/git/pub/jdhp-sap/snippets/ctapipe/plot_pixels_index.py
#!/usr/bin/env python3

"""
Display the pixel layout of the givent telescope as defined in a given simtel file.
"""

(pos_x_list, pos_y_list) = pixel_pos
pos_x_list = [float(pos.value) for pos in pos_x_list]
pos_y_list = [float(pos.value) for pos in pos_y_list]

assert len(pos_x_list) == len(pos_y_list)

# PLOT ####################################################################

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(13, 13))

# Scatter method ##################

ax.scatter(pos_x_list,    # x
           pos_y_list,    # y
           s=0.2,         # radius
           c="black",     # color
           alpha=0.75)

ax.plot(pos_x_list,    # x
        pos_y_list,    # y
        "k-",
        alpha=0.25)

for pixel_index in range(len(pos_x_list)):
    ax.text(pos_x_list[pixel_index], pos_y_list[pixel_index], str(pixel_index), fontsize=8)

ax.set_title("Pixels position (telescope {})".format(tel_id), fontsize=16)

ax.set_xlabel("x position (m)", fontsize=16)
ax.set_ylabel("y position (m)", fontsize=16)

#ax.set_xlim(-0.17, 0.17)
#ax.set_ylim(-0.17, 0.17)

plt.show()

### Plot image with ctapipe

In [ ]:
image = calibrated_image   # pe, uncalibrated_image, calibrated_image

In [ ]:
#%load ../../snippets/ctapipe/plot_events_image.py

import ctapipe
import ctapipe.visualization
from ctapipe.io.hessio import hessio_event_source

from matplotlib import pyplot as plt

x, y = ev.inst.pixel_pos[tel_id]
foclen = ev.inst.optical_foclen[tel_id]
geom = ctapipe.instrument.CameraGeometry.guess(x, y, foclen)

disp = ctapipe.visualization.CameraDisplay(geom,
                                           image=image,   # pe, adc_sum
                                           title='CT%d' % tel_id)
disp.enable_pixel_picker()
disp.add_colorbar()

disp.axes.set_title('Telescope {:03d} Event {}'.format(tel_id, event_id))

## 1D to 2D

In [ ]:
cam_id = "NectarCam"
rot = 0

geom = CameraGeometry.from_name(cam_id)

if geom.pix_type=='rectangular':
    print("Error: skip non-hexagonal cameras, since they don't need conversion")

In [ ]:
print(image.shape)
    
geom2d, image2d = geomconv.convert_geometry_1d_to_2d(geom,
                                                     image,
                                                     geom.cam_id + str(rot),
                                                     add_rot=0)

print(image2d.shape)
print(image2d)

disp = ctapipe.visualization.CameraDisplay(geom2d, image=image2d)
disp.enable_pixel_picker()
disp.add_colorbar()

### Get and print the pixel mask of the new geometry

In [ ]:
print(geom2d.mask.shape)
print(geom2d.mask)

disp = ctapipe.visualization.CameraDisplay(geom2d, image=geom2d.mask)
disp.enable_pixel_picker()
disp.add_colorbar()

### Put NaN in blank areas (instead 0)

This step is optional, the aim is to be closer to what is actually done in `datapipe`.

In [ ]:
image2d[np.logical_not(geom2d.mask)] = np.nan

print(image2d)

In [ ]:
disp = ctapipe.visualization.CameraDisplay(geom2d, image=image2d)
disp.enable_pixel_picker()
disp.add_colorbar()

### Inject noise in blank areas

In [ ]:
nan_mask = np.isnan(image2d)

print(np.nanmin(image2d))
print(np.nanmean(image2d))
print(np.nanstd(image2d))

# See https://stackoverflow.com/questions/29365194/replacing-missing-values-with-random-in-a-numpy-array

# GCT
#image2d[nan_mask] = np.random.poisson(lam=3, size=np.count_nonzero(nan_mask)) - 3.
image2d[nan_mask] = np.random.normal(0.13, 5.77, size=np.count_nonzero(nan_mask))

In [ ]:
#disp = ctapipe.visualization.CameraDisplay(geom2d, image=image2d)
#disp.enable_pixel_picker()
#disp.add_colorbar()

plt.imshow(image2d, interpolation="nearest", cmap="gnuplot2")
plt.colorbar()

## 2D to 1D

In [ ]:
geom1d, image1d = geomconv.convert_geometry_back(geom2d,
                                                 image2d,
                                                 geom.cam_id + str(rot),
                                                 add_rot=rot)

print(image.shape)

disp = ctapipe.visualization.CameraDisplay(geom1d, image=image1d) # Warning: "geom != geom1d" (becauseof the use fo np.histogram to make the conversino) thus use geom1d here, not geom !
disp.enable_pixel_picker()
disp.add_colorbar()